## RAG ##
Retrieval Augmented Generation
- Retrieval - Fetching data from a database or a large data source
- Augmented - To enhance. Brings together answers that have been retrieved by the retriever.
- Generation - Try to make sense of what it has been augmented and generate an output.

> - Vector databases - A vector database is a specialized type of database designed to store, index, and search vector embeddings—which are numerical representations of data like text, images, audio, or video.

##LLMs##
Grok, chatgpt



In [2]:
!pip install langchain-groq pypdf
!pip install chromadb
!pip install -U langchain-community



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached sqlalchemy-2.0.41-cp313-cp313-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.12.13-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached propcache-0.3.2-cp313-cp313-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp313-cp313-win_amd64.whl.metadata (76 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached greenlet-3.2.3-cp313-cp313-win_amd64.whl


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
!pip install sentence-transformers


  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.53.1-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.7.1-cp313-cp313-win_amd64.whl.metadata (28 kB)
Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
Using cached torch-2.7.1-cp313-cp313-win_amd64.whl (216.1 MB)
Using cached transformers-4.53.1-py3-none-any.whl (10.8 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
#import libraries
#LangChain
#Retrievers

from langchain.retrievers import ParentDocumentRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.storage import InMemoryStore
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
import os

c:\Users\NGARE\DS_Python_Lux\Ds_projects\FinanceBillRag\finance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
load_dotenv()
api_key = os.getenv("API_KEY")

if not api_key:
  raise ValueError('API key not loaded successful')
else:
  print('API key loaded successfully')

API key loaded successfully


In [19]:
pdf_link = r'C:\Users\NGARE\DS_Python_Lux\Ds_projects\FinanceBillRag\The Finance Bill 2025.pdf'

In [20]:
# Loading the PDF file
loader = PyPDFLoader(pdf_link)
docs = loader.load()

print(len(docs))

135


In [21]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 700)
parent_docs = parent_splitter.split_documents(docs)

print(len(parent_docs))
child_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 700)

205


In [22]:
#initializing the embedding model
bge_model = SentenceTransformer("BAAI/bge-base-en")

#creating the embedding class
class BGEEmbeddings:
  def embed_documents(self, text):
    """Generate Embeddings for batch of documents"""
    return bge_model.encode(text, batch_size = 8, normalize_embeddings = True).tolist()
  def embed_query(self, text):
    """Generate Embeddings for a single query"""
    return bge_model.encode([text],normalize_embeddings = True).tolist()[0]


from langchain.embeddings.base import Embeddings

class LangchainBGEEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return BGEEmbeddings().embed_documents(texts)

    def embed_query(self, text):
        return BGEEmbeddings().embed_query(text)
embedding = LangchainBGEEmbeddings()


c:\Users\NGARE\DS_Python_Lux\Ds_projects\FinanceBillRag\finance\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NGARE\.cache\huggingface\hub\models--BAAI--bge-base-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [23]:
# Setup Vector Store and Retriever

store = InMemoryStore()
vectorstore = Chroma(collection_name="kenya_finance_bill",
                     embedding_function=embedding,
                     persist_directory = 'finance_bill_vectorstore')

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter
)

C:\Users\NGARE\AppData\Local\Temp\ipykernel_13068\534723420.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="kenya_finance_bill",


In [24]:
#Add Documents to the Retriever

retriever.add_documents(parent_docs)
print("Documents indexed into retriever")

Documents indexed into retriever


In [25]:
# Initialize the Groq LLM

llm = ChatGroq(
    temperature=0.0,
    model_name="llama3-70b-8192",
    api_key=api_key
)


In [26]:
#adding memory to the RAG
# A temperature of 0.0 means the LLM will produce the most deterministic and consistent (least "creative") output
# The memory variable will store the conversational memory component.
# The conversation variable will hold the core RAG conversational chain
llm = ChatGroq(temperature = 0.0, model_name = "llama3-70b-8192", api_key = api_key)
memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True
)
conversation = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriever,
    memory = memory,
    verbose = True
)

C:\Users\NGARE\AppData\Local\Temp\ipykernel_13068\2729261503.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [27]:
#(Optional) Custom Prompt Template

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an expert assistant. Use the following context to answer the question.
If the answer is not in the context, say "I can only answer questions related to Kenya finance bill 2025."

Context:
{context}

Question:
{question}

Answer:"""
)

In [28]:
# #Create RetrievalQA Chain

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)



In [29]:

#Ask a Question

query = "What are the proposed tax changes in the Finance Bill 2025?"
response = qa_chain.run(query)
print("Answer:", response)

C:\Users\NGARE\AppData\Local\Temp\ipykernel_13068\1491727289.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Answer: Based on the provided context, the proposed tax changes in the Finance Bill 2025 include:

1. Amendments to the Income Tax Act (Cap. 470), Value Added Tax Act (Cap. 476), Excise Duty Act (Cap. 472), Tax Procedures Act (Cap. 469B), and Miscellaneous Fees and Levies Act (Cap. 469C).

2. Exemption from stamp duty on the transfer of property by a company to its shareholders as part of an internal reorganisation.

3. Introduction of electronic tax invoices, which must contain specific information such as the supplier's and purchaser's details, description of the supply, consideration, tax rate, and total tax amount.

4. Penalties for late submission and failure to submit returns, including:
   - 25% of the tax due or KES 10,000 for employment income returns submitted late
   - KES 1,000 for Turnover Tax returns submitted late
   - 5% of the tax payable or KES 10,000 for Value Added Tax and Excise Duty returns submitted late
   - 5% of the tax payable or KES 20,000 for other returns 

In [61]:
#Question 2

query = "What are the proposed income tax bands or rates?"
response = qa_chain.run(query)
print("Answer:", response)


Answer: According to the context, the proposed income tax bands or rates are:

For individuals:

* 10% on the first KSh. 288,000
* 25% on the next KSh. 100,000
* 30% on the next KSh. 5,612,000
* 32.5% on the next KSh. 3,600,000
* 35% on all income over KSh. 9,600,000

For resident companies:

* 9.00 for the year of income 1974 and each subsequent year of income up to and including the year of income 1990
* 8.50 for the year of income 1990
* 8.00 for the year of income 1991
* 7.50 for the year of income 1992
* 7.00 for the year of income 1993 up to and including the year of income 1997


In [55]:
#Question 3

query = "How does the bill affect small businesses or startups?"
response = qa_chain.run(query)
print("Answer:", response)

Answer: According to the context, the bill affects small businesses or startups in the following ways:

* Small businesses or small-scale farmers with an annual turnover of less than five million shillings are exempt from issuing electronic tax invoices, but the purchaser shall issue a tax invoice for the purpose of ascertaining tax liability (Section 23A, subsection 3A).
* Non-resident persons with an annual turnover of less than five million shillings are exempt from certain tax provisions (Section 12G, subsection 4(d)).

These provisions seem to provide some relief to small businesses and startups by reducing their compliance burden and tax liability.


In [62]:
conversation("What are the proposed tax changes in the Finance Bill 2025?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are the proposed tax changes in the Finance Bill 2025?
Assistant: Based on the provided context, the proposed tax changes in the Finance Bill 2025 include:

1. Amendments to the Income Tax Act (Cap. 470), Value Added Tax Act (Cap. 476), Excise Duty Act (Cap. 472), Tax Procedures Act (Cap. 469B), and Miscellaneous Fees and Levies Act (Cap. 469C).
2. Exemption from stamp duty on the transfer of property by a company to its shareholders as part of an internal reorganisation.
3. Introduction of electronic tax invoices, which must contain specific information such as the supplier's and purchaser's details, serial number, date and time of issue, description of supply, tax rate, and total tax amount.
4. Requirement for businesses to issue electronic tax invoices 

{'question': 'What are the proposed tax changes in the Finance Bill 2025?',
 'chat_history': [HumanMessage(content='What are the proposed tax changes in the Finance Bill 2025?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Based on the provided context, the proposed tax changes in the Finance Bill 2025 include:\n\n1. Amendments to the Income Tax Act (Cap. 470), Value Added Tax Act (Cap. 476), Excise Duty Act (Cap. 472), Tax Procedures Act (Cap. 469B), and Miscellaneous Fees and Levies Act (Cap. 469C).\n2. Exemption from stamp duty on the transfer of property by a company to its shareholders as part of an internal reorganisation.\n3. Introduction of electronic tax invoices, which must contain specific information such as the supplier's and purchaser's details, serial number, date and time of issue, description of supply, tax rate, and total tax amount.\n4. Requirement for businesses to issue electronic tax invoices through a system established by the Commissioner, a

In [30]:
conversation("What are the proposed income tax bands or rates?")


C:\Users\NGARE\AppData\Local\Temp\ipykernel_13068\26559599.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversation("What are the proposed income tax bands or rates?")




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
418 	 The Finance Bill, 2025 
investment value for the preceding four years from the date that this 
provision comes into force or the cumulative investment for the 
succeeding three years outside Nairobi City County or Mombasa County 
is at least one billion shillings; 
(b) the investment value outside Nairobi City County and Mombasa 
County in that year of income is at least two hundred and fifty 
million shillings; or 
(c) the person has incurred investment in a special economic zone. 
(113) Paragraph (1A) shall apply to items listed under paragraphs 1(1)(a)(i) 
and (ii), and (1) (b) (i). 
Third Schedule of Cap. 470, it is proposed to amend— 
THIRD SCHEDULE [ss. 29, 30, 31, 32, 33, 34 and 35] 
RAT

{'question': 'What are the proposed income tax bands or rates?',
 'chat_history': [HumanMessage(content='What are the proposed income tax bands or rates?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='According to the provided context, the proposed income tax bands or rates are:\n\n1. Individual rates of tax:\n\t* 10% on the first KSh. 288,000\n\t* 25% on the next KSh. 100,000\n\t* 30% on the next KSh. 5,612,000\n\t* 32.5% on the next KSh. 3,600,000\n\t* 35% on all income over KSh. 9,600,000\n2. Corporate rate of tax:\n\t* For resident companies:\n\t\t+ 9.00 for the year of income 1974 and each subsequent year of income up to and including the year of income 1990\n\t\t+ 8.50 for the year of income 1990\n\t\t+ 8.00 for the year of income 1991\n\t\t+ 7.50 for the year of income 1992\n\t\t+ 7.00 for the year of income 1993 up to and including the year of income 1997', additional_kwargs={}, response_metadata={})],
 'answer': 'According to the provided context, the prop

In [31]:
conversation("")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are the proposed income tax bands or rates?
Assistant: According to the provided context, the proposed income tax bands or rates are:

1. Individual rates of tax:
	* 10% on the first KSh. 288,000
	* 25% on the next KSh. 100,000
	* 30% on the next KSh. 5,612,000
	* 32.5% on the next KSh. 3,600,000
	* 35% on all income over KSh. 9,600,000
2. Corporate rate of tax:
	* For resident companies:
		+ 9.00 for the year of income 1974 and each subsequent year of income up to and including the year of income 1990
		+ 8.50 for the year of income 1990
		+ 8.00 for the year of income 1991
		+ 7.50 for the year of income 1992
		+ 7.00 for the year of income 1993 up to and including the year of income 1997
Follow Up Input: 
Standalone question:

> Finished chain.


> Ente

{'question': '',
 'chat_history': [HumanMessage(content='What are the proposed income tax bands or rates?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='According to the provided context, the proposed income tax bands or rates are:\n\n1. Individual rates of tax:\n\t* 10% on the first KSh. 288,000\n\t* 25% on the next KSh. 100,000\n\t* 30% on the next KSh. 5,612,000\n\t* 32.5% on the next KSh. 3,600,000\n\t* 35% on all income over KSh. 9,600,000\n2. Corporate rate of tax:\n\t* For resident companies:\n\t\t+ 9.00 for the year of income 1974 and each subsequent year of income up to and including the year of income 1990\n\t\t+ 8.50 for the year of income 1990\n\t\t+ 8.00 for the year of income 1991\n\t\t+ 7.50 for the year of income 1992\n\t\t+ 7.00 for the year of income 1993 up to and including the year of income 1997', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Based on th